<a href="https://colab.research.google.com/github/diqnfl777/2022F-Ajou-ML-TEAM3/blob/main/LibraryModelFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT finetuning

In [1]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 27.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-fxgx5ozl
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-fxgx5ozl
     |████████████████████████████████| 129 kB 31.1 MB/s 
     |████████████████████████████████| 344 kB 65.8 MB/s 
     |████████████████████████████████| 54.7 MB 89.2 MB/s 
     |████████████████████████████████| 4.5 MB 49.8 MB/s 
     |████████████████████████████████| 1.2 MB 84.9 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:43tcmalloc: large alloc 1147494400 bytes == 0x3a804000 @  0x7fe3cc86a615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4

#Import 및 사전준비

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [2]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
from transformers import AutoTokenizer
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [40]:
## GPU
device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# 데이터 불러오기

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
#데이터 가공
#청구기호 숫자 앞 2개만 따와서 각각 매핑. 65~70은 예외적으로 경영학
import pandas as pd
import re 

data = pd.read_csv("/content/gdrive/MyDrive/LibraryCsv/indexhapbon.csv",encoding='UTF8',usecols = ['제목', '청구번호','목차'])
data = data.loc[:,['제목', '청구번호','목차']]
data = data.dropna()
data = data.drop_duplicates()


data['청구번호'] = data['청구번호'].apply(str)

data['청구번호'] = data['청구번호'].replace({r'(.*?)(\d{2})\d.*' : r'\2'}, regex=True)

data['청구번호'] = pd.to_numeric(data['청구번호'],errors = 'coerce')
data = data.dropna()
data.loc[(data['청구번호'] >= 100), ['청구번호']] = None
data = data.dropna()
data.loc[(data['청구번호'] < 10), ['청구번호']] = 1000
data.loc[(data['청구번호'] < 20), ['청구번호']] = 1001
data.loc[(data['청구번호'] < 30), ['청구번호']] = 1002
data.loc[(data['청구번호'] < 40), ['청구번호']] = 1003
data.loc[(data['청구번호'] < 50), ['청구번호']] = 1004
data.loc[(data['청구번호'] < 60), ['청구번호']] = 1005
data.loc[(data['청구번호'] < 65), ['청구번호']] = 1006 #경영학쪽은 따로 분류이기 떄문에 65 사용
data.loc[(data['청구번호'] < 70), ['청구번호']] = 1007 #즉, 1007 쪽이 경영학책
data.loc[(data['청구번호'] < 80), ['청구번호']] = 1008
data.loc[(data['청구번호'] < 90), ['청구번호']] = 1009
data.loc[(data['청구번호'] <  100), ['청구번호']] = 1010

labels = {'0':'총류',
          '1':'철학',
          '2':'종교',
          '3':'사회학',
          '4':'언어',
          '5':'자연과학',
          '6':'기술과학',
          '7':'경영학',
          '8':'예술',
          '9':'문학',
          '10':'역사'
          }

data['청구번호'] = data['청구번호']%1000
data['청구번호'] = data['청구번호'].astype(int)
regex = '[0-9|A-Z|a-z|ㄱ-ㅎ|ㅏ-ㅣ|가-힣|\s]*[ㄱ-ㅎ|ㅏ-ㅣ|가-힣][0-9|A-Z|a-z|ㄱ-ㅎ|ㅏ-ㅣ|가-힣|\s]*'
data

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,제목,청구번호,목차
0,어디서 무엇이 되어 다시 만나랴,9,\n\n결혼식 ㅣ 아기 ㅣ 신혼기 ㅣ 아내의 머리칼 ㅣ 아...
1,나의 사랑 클레멘타인,9,\n\n결혼식 ㅣ 아기 ㅣ 신혼기 ㅣ 아내의 머리칼 ㅣ 아...
2,초파리를 알면 유전자가 보인다,5,서문 \r\n 서론 \r\n제1장 지도\r\n제2장 ...
3,아웃사이더의 반란,3,한국어판 서문 / 5\r\n 프롤로그 / 13\r\n제1장 보수...
4,브랜드 인문학 :잠재된 표현 욕망을 깨우는 감각 수업,7,"1부-정체성\r\n1 프라다: 나의 정체성을 알 때, 비로소 브랜드는 ‘필..."
...,...,...,...
71533,한국 시극사 연구,9,ㆍ책머리에 \r\n\r\n제1장 서론 \r\n\r\n1. 연구 목적…17 \r\n2...
71534,조선의 서정시인 퇴계 이황 : 우리가 몰랐던 퇴계의 남도여행,9,"ㆍ책머리에\r\n\r\n1. 퇴계의 시혼, 남도에서 피다\r\n퇴계가 남쪽으로 간 ..."
71535,"내면기행 : 선인들, 스스로 묘비명을 쓰다",10,ㆍ책을 엮으며\r\n\r\n제1부 이사람을 보라\r\n제2부 이것으로 만족이다\r\...
71536,뉴 스토리 뉴 스타일,0,ㆍ책을 펴내며\r\n\r\n서 장 새로움을 위해\r\n제1장 살인의 추억\r\n제2...


In [9]:
data_sample = data.sample(frac = 0.05, random_state = 5)
df_train, df_val, df_test = np.split(data_sample.sample(frac=1, random_state=42), 
                                     [int(.8*len(data_sample)), int(.9*len(data_sample))])

# DataSets

In [17]:
from tokenizers import Tokenizer
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
en_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = get_tokenizer()
kr_tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [18]:
class EN_Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels =  df['청구번호'].values
        self.dic = [en_tokenizer(text, padding='max_length', max_length = 64, truncation=True, return_tensors="pt") for text in df['제목']]
    
    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def __getitem__(self, idx):

        batch_texts = self.dic[idx]
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [19]:
class KR_BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [20]:
class Library_Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.title_data = []
        transform = nlp.data.BERTSentenceTransform(                                 # 한글용
            kr_tok, max_seq_length=64, pad=True, pair=False)
        for text in df['제목']:
          if re.fullmatch(regex, text):
            self.title_data.append((np.int32(1),transform(text)))
          else:
            self.title_data.append((np.int32(0), en_tokenizer(text, padding='max_length', max_length = 64, truncation=True, return_tensors="pt")))
        self.index_data = [en_tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in df['목차']]
        self.labels = [np.int32(i) for i in df['청구번호']]
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        lang, item = self.title_data[idx]
        return (lang, item ,self.labels[idx], self.index_data[idx])

In [21]:
sampledata = data[:100]

In [22]:
sampledata

,제목,청구번호,목차
0,어디서 무엇이 되어 다시 만나랴,9,\n\n결혼식 ㅣ 아기 ㅣ 신혼기 ㅣ 아내의 머리칼 ㅣ 아...
1,나의 사랑 클레멘타인,9,\n\n결혼식 ㅣ 아기 ㅣ 신혼기 ㅣ 아내의 머리칼 ㅣ 아...
2,초파리를 알면 유전자가 보인다,5,서문 \r\n 서론 \r\n제1장 지도\r\n제2장 ...
3,아웃사이더의 반란,3,한국어판 서문 / 5\r\n 프롤로그 / 13\r\n제1장 보수...
4,브랜드 인문학 :잠재된 표현 욕망을 깨우는 감각 수업,7,"1부-정체성\r\n1 프라다: 나의 정체성을 알 때, 비로소 브랜드는 ‘필..."
...,...,...,...
95,수학의 배신,5,1장 거대한 착각 8\r\n 2장 무엇을 위해 수학을 공부하는가 24\r\...
96,(손에 잡히는) 인터넷 비지니스 전략,7,1장 인터넷 이해하기\n 1. 미래로 가는 길 인터넷\n 2. 인터넷 혁명\n 3...
97,마리의 돼지의 낙타 :엄우흠 장편소설,9,1장\r\n 영혼이 없는 떡볶이 \r\n\r\n 2장\r\n 대지의 비늘 \r\n...
98,체리레몬칵테일 :김규나 장편소설,9,1장_7\r\n 2장_50\r\n 3장_121\r\n 4장_211\r\n 5장_3...


In [23]:
ld = Library_Dataset(sampledata)

#Models

In [50]:
def load_checkpoint(load_path, model):# ------ save_checkpoint 함수에서 저장된 모델을 가져오기.
    if load_path == None:
        return
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

In [51]:
ML_load_path = "/content/gdrive/MyDrive/Learned/fullLearn.multiLangModel"
KR_load_path = "/content/gdrive/MyDrive/Learned/res4.koBertModel"
ID_load_path = "/content/gdrive/MyDrive/Learned/indexmodel2.multiLangModel"

In [52]:
from transformers import BertModel

class ML_BertClassifier(nn.Module):

    def __init__(self, dropout=0.7):
        super(ML_BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 11)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        #final_layer = self.relu(linear_output)
        return linear_output
ML_model = ML_BertClassifier()
load_checkpoint(ML_load_path, ML_model)
ML_model = ML_model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from <== /content/gdrive/MyDrive/Learned/fullLearn.multiLangModel


In [53]:
class KR_BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=11,
                 dr_rate=None,
                 params=None):
        super(KR_BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate          
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

dr_rate = 0.7
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")
KR_model = KR_BERTClassifier(bertmodel, dr_rate=dr_rate)
load_checkpoint(KR_load_path, KR_model)
KR_model = KR_model.cuda()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Model loaded from <== /content/gdrive/MyDrive/Learned/res4.koBertModel


In [54]:
class Index_BertClassifier(nn.Module):
    def __init__(self, dropout=0.75):
        super(Index_BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 11)
        self.relu = nn.ReLU()
    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        return linear_output
ID_model = Index_BertClassifier()
load_checkpoint(ID_load_path,ID_model)
ID_model = ID_model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from <== /content/gdrive/MyDrive/Learned/indexmodel2.multiLangModel


In [55]:
'''
class LibraryModel(nn.Module):
    def __init__(self, ML_model, KR_model,ID_model):
        super(LibraryModel, self).__init__()
        self.ML_model = ML_model
        self.ID_model = ID_model
        self.KR_model = KR_model
        self.classifier = nn.Linear(22, 11)
        
    def forward(self, title, index, lang):
        if(lang == 1):
            token_ids, valid_length, segment_ids = title
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length.to(device)
            output1 = self.KR_model(token_ids, valid_length, segment_ids)
        else: 
            title_mask = title['attention_mask'].to(device)
            title_id = title['input_ids'].squeeze(1).to(device)
            output1 = self.ML_model(title_id, title_mask)
        index_mask = index['attention_mask'].to(device)
        index_id = index['input_ids'].squeeze(1).to(device)
        output2 = self.ID_model(index_id, index_mask)
        x = torch.cat((output1, output2), dim=1)
        x = self.classifier(F.relu(x))
        return x
Library_model = LibraryModel(ML_model, KR_model,ID_model)
'''

"\nclass LibraryModel(nn.Module):\n    def __init__(self, ML_model, KR_model,ID_model):\n        super(LibraryModel, self).__init__()\n        self.ML_model = ML_model\n        self.ID_model = ID_model\n        self.KR_model = KR_model\n        self.classifier = nn.Linear(22, 11)\n        \n    def forward(self, title, index, lang):\n        if(lang == 1):\n            token_ids, valid_length, segment_ids = title\n            token_ids = token_ids.long().to(device)\n            segment_ids = segment_ids.long().to(device)\n            valid_length= valid_length.to(device)\n            output1 = self.KR_model(token_ids, valid_length, segment_ids)\n        else: \n            title_mask = title['attention_mask'].to(device)\n            title_id = title['input_ids'].squeeze(1).to(device)\n            output1 = self.ML_model(title_id, title_mask)\n        index_mask = index['attention_mask'].to(device)\n        index_id = index['input_ids'].squeeze(1).to(device)\n        output2 = self.ID_m

# 최종 모델

In [24]:
from torch.optim import AdamW
from tqdm import tqdm
from transformers.optimization import get_cosine_schedule_with_warmup

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Library_Dataset(train_data), Library_Dataset(val_data)
    warmup_rate = 0.1
    batch_size=1
    num_total_steps = (len(df_train) // batch_size) * epochs
    num_warmup_steps = num_total_steps * warmup_rate
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True,num_workers=1)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr= learning_rate)
    scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                         num_warmup_steps=num_warmup_steps, 
                                                         num_training_steps=num_total_steps)
    if use_cuda:
            print("using cuda!")
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            iloss = 0

            for i, dataget in enumerate(tqdm(train_dataloader)) :
                lang, train_title, train_label, train_index = dataget
                train_label = train_label.to(device)
                output = model(train_title, train_index, lang)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                iloss += batch_loss.item()
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
                scheduler.step()
                if(i%100 == 99):
                  print(i//100, "loss value", iloss/100);
                  iloss =0
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():
                for val_dataget in val_dataloader:
                    val_lang, val_title, val_label, val_index = dataget
                    val_label = val_label.to(device)
                    output = model(val_title, val_index, val_lang)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .4f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .4f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}') 
                  
EPOCHS = 1
model = LibraryModel(ML_model, KR_model,ID_model)
LR = 0.0000003          
train(model, df_train, df_val, LR, EPOCHS)
LB_model = model

using cuda!


  3%|▎         | 100/2859 [00:28<13:17,  3.46it/s]

0 loss value 2.423138613700867


  7%|▋         | 200/2859 [00:57<12:54,  3.43it/s]

1 loss value 2.4381912726163866


...

18 loss value 2.0717538774013518


 70%|██████▉   | 2000/2859 [09:46<04:10,  3.43it/s]

19 loss value 2.0547488814592363


 73%|███████▎  | 2080/2859 [10:10<03:48,  3.41it/s]


KeyboardInterrupt: ignored

In [31]:
'''
def evaluate(model, test_data):
    test = Library_Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
    total_acc_test = 0
    with torch.no_grad():
        for dataget in tqdm(test_dataloader):
            lang, test_title, test_label, test_index = dataget
            test_label = test_label.to(device)
            output = model(test_title, test_index, lang)
            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc  
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
evaluate(model, df_test)
'''

100%|██████████| 358/358 [00:16<00:00, 21.76it/s]

Test Accuracy:  0.517


# SAVE

In [32]:
def save_checkpoint(save_path, model, valid_loss):# ------ 모델 평가를 위해 훈련 과정을 저장
    if save_path == None:
        return
    state_dict = {'model_state_dict': model.state_dict(), 'valid_loss': valid_loss}
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


In [33]:
save_checkpoint("/content/gdrive/MyDrive/ver1.finalModel", model, 1.96)

Model saved to ==> /content/gdrive/MyDrive/ver1.finalModel


# TEST

In [61]:
def get_top_values(title, index = 0):
  #use_cuda = torch.cuda.is_available()
  #device = torch.device("cuda" if use_cuda else "cpu")
  #if use_cuda:
  #    EN_model = EN_model.cuda()
  with torch.no_grad():
    if re.fullmatch(regex, title):
      transform = nlp.data.BERTSentenceTransform(                                 # 한글용
        kr_tok, max_seq_length=64, pad=True, pair=False)
      token_ids, valid_length, segment_ids = transform(title)
      print(token_ids)
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      out = KR_model(token_ids, valid_length, segment_ids)
      print("kr")
    else:
      title_data = en_tokenizer(title, padding='max_length', max_length = 64, truncation=True, return_tensors="pt")
      input_id = title_data['input_ids']
      input_id = input_id.to(device)
      mask = title_data['attention_mask']
      mask = mask.to(device)
      out = ML_model(input_id, mask)
      print("ml")
    output = F.softmax(out[0],dim=-1)
    print("max for title: ",torch.topk(output,3))
    if(index != 0):
      index_data = en_tokenizer(index, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
      input_id = index_data['input_ids']
      input_id = input_id.to(device)
      mask = index_data['attention_mask']
      mask = mask.to(device)
      out = ID_model(input_id, mask)
      output = F.softmax(out[0],dim=-1)
      print("max for index: ",torch.topk(output,3))


In [62]:
get_top_values("언어의 심리학") # 확인해보고 싶은 제목 입력
labels

[   2 3241    3    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1]


AttributeError: ignored

In [ ]:
from pandas import DataFrame

toselftest = ["테스트1","테스트2","테스트3"]

labels = {0:'총류',
          1:'철학',
          2:'종교',
          3:'사회학',
          4:'언어',
          5:'자연과학',
          6:'기술과학',
          7:'경영학',
          8:'예술',
          9: '문학',
          10:'역사'
}

selftest = DataFrame({
  'value': toselftest,
  'null': [0 for x in toselftest]
})

selftest.to_csv("/content/gdrive/MyDrive/LibraryCsv/selftest.txt", sep="\t")
dataset_selftest = nlp.data.TSVDataset("/content/gdrive/MyDrive/LibraryCsv/selftest.txt", field_indices=[1,2], num_discard_samples=1)
data_selftest = BERTDataset(dataset_selftest, 0, 1, tok, max_len, True, False)
selftest_dataloader = torch.utils.data.DataLoader(data_selftest, batch_size=1, num_workers=1)

for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(selftest_dataloader), total=len(selftest_dataloader)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)

      prediction = out.cpu().detach().numpy()[0]
      minval = prediction.min()
      prediction = [x-minval for x in prediction]
      sumval = sum(prediction)
      print(toselftest[batch_id])
      for index, score in sorted(enumerate(prediction), reverse = True, key = lambda prediction:prediction[1]):
        print("{} {}%".format(labels[index], score*100/sumval))
